
# 1. Bibliotecas

In [0]:
from pyspark.sql.functions import col, when, expr, lit
from pyspark.sql.types import DecimalType, DateType, IntegerType
from datetime import datetime

---

# 2. Carregando dados

Utilizado sql para usar unity catalog

In [0]:
df_orders = spark.table('datum.silver.olist_orders')
df_orders_diff = spark.table('datum.silver.olist_orders_datediff')

df_orders = df_orders.join(df_orders_diff, (df_orders.order_id == df_orders_diff.order_id) & (df_orders.customer_id == df_orders_diff.customer_id), how = 'inner').where(col('order_status') == 'delivered')

del df_orders_diff

---
# 3. Análise


## 3.1 Agrupamento por categorização


Assumindo as seguinte política para entrega

| intervalo em dias | gravidade | sigla |
| ----------------- | --------- | ----- |
| 0 a - 5 dias | bom | B |
| -6 a -12 dias | ótimo | O |
| inferior a 12 dias | excelente | E |
| 1 a 3 dias | tolerável | T |
| 4 a 7 dias | ruim | R |
| 8 a 14 dias | grave | G |
| 15 a 28 dias | péssimo | P | 
| superior a 28 dias | urgente | U |

In [0]:
df_orders = (df_orders.withColumn('analise_entrega',
                  when((col('delivery_diff_promissed') <= 0) & (col('delivery_diff_promissed') >= -5), 'B | BOM').
                  when((col('delivery_diff_promissed') <= -6) & (col('delivery_diff_promissed') >= -12), 'O | OTIMO').
                  when(col('delivery_diff_promissed') < -12, 'E | EXCELENTE').
                  when((col('delivery_diff_promissed') >= 1) & (col('delivery_diff_promissed') <= 3), 'T | TOLERAVEL').
                  when((col('delivery_diff_promissed') >= 4) & (col('delivery_diff_promissed') <= 7), 'R | RUIM').
                  when((col('delivery_diff_promissed') >= 8) & (col('delivery_diff_promissed') <= 14), 'G | GRAVE').
                  when((col('delivery_diff_promissed') >= 15) & (col('delivery_diff_promissed') <= 28), 'P | PESSIMO').
                  when(col('delivery_diff_promissed') > 28, 'U | URGENTE')
                  ))

df_orders_sigla = df_orders.groupBy('analise_entrega').count().withColumn('porcentagem', (col('count')/df_orders.count() * 100).cast(DecimalType(5,2)))
df_orders_sigla = (df_orders_sigla.withColumnRenamed('analise_entrega', 'sigla').
                                   withColumnRenamed('count', 'ordens').
                                   withColumn('date_ref_carga', lit(datetime.now()).cast(DateType())).
                                   withColumn('ordens', col('ordens').cast(IntegerType())))

del df_orders

In [0]:
df_orders_sigla.display()

sigla,ordens,porcentagem,date_ref_carga
U | URGENTE,399,0.41,2024-04-14
O | OTIMO,31737,32.90,2024-04-14
E | EXCELENTE,48028,49.79,2024-04-14
T | TOLERAVEL,1870,1.94,2024-04-14
B | BOM,10171,10.54,2024-04-14
P | PESSIMO,985,1.02,2024-04-14
G | GRAVE,1478,1.53,2024-04-14
R | RUIM,1802,1.87,2024-04-14



Acima é realizado uma análise agrupando e categorizando as datas de entrega realizadas na categoria juntamente com o volume de pedidos e porcentagem proporcional de cada sigla


## 3.2 Agrupamento por categorização e estado

In [0]:
%sql
WITH 
customer_state AS (
  SELECT DISTINCT customer_id, customer_state
  FROM datum.silver.olist_customers),
orders AS (
  SELECT customer_id, 
         delivery_diff_promissed,
         CASE 
          WHEN delivery_diff_promissed <= 0 AND  delivery_diff_promissed >= -5 THEN 'B'
          WHEN delivery_diff_promissed <= -6 AND delivery_diff_promissed >= -12 THEN 'O'
          WHEN delivery_diff_promissed < -12 THEN  'E'
          WHEN delivery_diff_promissed >= 1 AND  delivery_diff_promissed <= 3 THEN'T'
          WHEN delivery_diff_promissed >= 4 AND  delivery_diff_promissed <= 7 THEN'R'
          WHEN delivery_diff_promissed >= 8 AND  delivery_diff_promissed <= 14 THEN 'G'
          WHEN delivery_diff_promissed >= 15 AND delivery_diff_promissed <= 28 THEN 'P'
          WHEN delivery_diff_promissed > 28 THEN  'B'
          END
          AS categoria_entrega
  FROM datum.silver.olist_orders_datediff
),
orders_estado_proporcional AS (
  SELECT customer_state AS estado, categoria_entrega, ROUND(COUNT(categoria_entrega)) AS proporcional
  FROM orders 
    LEFT JOIN customer_state 
      ON orders.customer_id = customer_state.customer_id
  GROUP BY customer_state, categoria_entrega
)

SELECT estado, 
       CASE WHEN categoria_entrega = 'B' THEN 'B | BOM'
            WHEN categoria_entrega = 'O' THEN 'O | OTIMO'
            WHEN categoria_entrega = 'E' THEN 'E | EXCELENTE'
            WHEN categoria_entrega = 'T' THEN 'T | TOLERAVEL'
            WHEN categoria_entrega = 'R' THEN 'R | RUIM'
            WHEN categoria_entrega = 'G' THEN 'G | GRAVE'
            WHEN categoria_entrega = 'P' THEN 'P | PESSIMO'
            WHEN categoria_entrega = 'U' THEN 'U | URGENTE'
            END AS sigla, 
       CAST(proporcional AS INT) AS ordens, 
       CAST(proporcional / SUM(proporcional) OVER(PARTITION BY estado ORDER BY estado) *100 AS DECIMAL(5,2)) AS porcentagem,
       CAST(NOW() AS DATE) AS date_ref_carga
FROM orders_estado_proporcional AS oep
ORDER BY estado, proporcional DESC, sigla

estado,sigla,ordens,porcentagem,date_ref_carga
AC,E | EXCELENTE,65,81.25,2024-04-14
AC,O | OTIMO,8,10.00,2024-04-14
AC,B | BOM,5,6.25,2024-04-14
AC,P | PESSIMO,1,1.25,2024-04-14
AC,T | TOLERAVEL,1,1.25,2024-04-14
AL,E | EXCELENTE,170,42.82,2024-04-14
AL,O | OTIMO,82,20.65,2024-04-14
AL,B | BOM,64,16.12,2024-04-14
AL,R | RUIM,29,7.30,2024-04-14
AL,G | GRAVE,24,6.05,2024-04-14



Por questão de tempo a análise está inicialmente em SQL para simplificar lógica usada, futuramente podendo ser migrada para spark.

A regra agora informa o estado seguido da sigla e voluma de ocorrencias em caso estado e sigla assim como a porcentagem proporcional pelo total do estado,


# 4. Salvando tabelas

In [0]:
%sql

USE CATALOG datum

In [0]:
%sql

USE DATABASE gold

In [0]:
%sql

CREATE TABLE IF NOT EXISTS olist_entregas_estado
(
  sigla         STRING NOT NULL,
  ordens        INTEGER,
  porcentagem   DECIMAL(5,2),
  date_ref_carga DATE
)
USING DELTA
LOCATION 'abfss://unity-datum@datumunity.dfs.core.windows.net/gold/olist_entregas_estado'
PARTITIONED BY(date_ref_carga, sigla)

In [0]:
%sql

CREATE TABLE IF NOT EXISTS olist_entregas_estado_sigla
(
  estado         STRING NOT NULL,
  sigla          STRING NOT NULL,
  ordens         INTEGER,
  porcentagem    DECIMAL(5,2),
  date_ref_carga DATE
)
USING DELTA
LOCATION 'abfss://unity-datum@datumunity.dfs.core.windows.net/gold/olist_entregas_estado_sigla'
PARTITIONED BY(date_ref_carga, estado)

In [0]:
if df_orders_sigla.count() != 0 and df_orders_sigla is not None:
    df_orders_sigla.write.format('delta').mode('overwrite').save('abfss://unity-datum@datumunity.dfs.core.windows.net/gold/olist_entregas_estado')
del df_orders_sigla    

In [0]:
if _sqldf.count() != 0 and _sqldf is not None:
    _sqldf.write.format('delta').mode('overwrite').save('abfss://unity-datum@datumunity.dfs.core.windows.net/gold/olist_entregas_estado_sigla')
del _sqldf